In [1]:
import pandas as pd
import openai
import time
from src.utils import *

import warnings
warnings.filterwarnings('ignore')

c:\Users\RodrigoMartínezAlons\anaconda3\Lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


### 1) Data

In [2]:
df_bayer=pd.read_csv('data/bayer_whole_cat.csv')
df_lfa=pd.read_excel('data/MRO_Manufacturer_LFA1_2025_01_27.xlsx')

In [3]:
with open("config.json", "r") as file:
    config = json.load(file)
openai.api_key = config["openai_api_key"]

In [4]:
df_lfa

,OPSYS,LIFNR,LAND1,NAME1,ADRNR,MCOD1,KTOkK,LOEVM
0,PMD,5368080,KY,AAA Technology & Specialties Co,445840,AAA TECHNOLOGY & SPECIALT,YCMP,NaN
1,PMD,5368099,KY,ABCO PLASTICS DIV,445835,ABCO PLASTICS DIV,YCMP,NaN
2,PMD,5368102,KY,Pikotek/AccuSeal,445813,PIKOTEK/ACCUSEAL,YCMP,NaN
3,PMD,5368110,KY,"ACF INDUSTRIES, INC.",445841,"ACF INDUSTRIES, INC.",YCMP,NaN
4,PMD,5368129,KY,Atlantic States Cast Iron Pipe C,445777,ATLANTIC STATES CAST IRON,YCMP,NaN
...,...,...,...,...,...,...,...,...
10978,PMD,8562020,KY,CORROSION PRODUCTS OF TEXAS,1321945,CORROSION PRODUCTS OF TEX,YCMP,NaN
10979,PMD,8562055,KY,"FUJIKIN OF AMERICA,INC.",1321946,"FUJIKIN OF AMERICA,INC.",YCMP,NaN
10980,PMD,8568363,KY,"TRD MANUFACTURING,INC.",1324328,"TRD MANUFACTURING,INC.",YCMP,NaN
10981,PMD,8575734,KY,COOPER WIRING DEVICES,1325920,COOPER WIRING DEVICES,YCMP,NaN


In [5]:
df_bayer

,product_id,quality_status,manufacturer_name,manufacturer_pid,model,short_description,long_description,additional_description,noun,norm,scope,site
0,101050487,SPARROW_APPROVED,CKD,SCM-FA-20B-100-T3H3-D-Z-6,SCM-FA-20B-100-T3H3-D-Z-6,air cylinder,NaN,NaN,cylinder,NaN,In,YA2Z Shiga
1,101050486,SPARROW_APPROVED,CKD,SCM-TA-25B-50-T3H3-D-B2-6,SCM-TA-25B-50-T3H3-D-B2-6,air cylinder,NaN,NaN,cylinder,NaN,In,YA2Z Shiga
2,101050485,SPARROW_APPROVED,CKD,SCM-CA-32B-100-T3H3-D-Y-8,SCM-CA-32B-100-T3H3-D-Y-8,air cylinder,NaN,NaN,cylinder,NaN,In,YA2Z Shiga
3,101050484,SPARROW_APPROVED,CKD,SSD-L-160-30-T3H3-D-N-6-IN,SSD-L-160-30-T3H3-D-N-6-IN,air cylinder,NaN,NaN,cylinder,NaN,In,YA2Z Shiga
4,101050483,SPARROW_APPROVED,CKD,SSD-KL-DB-16-40-K3H3-D-N-6-IN,SSD-KL-DB-16-40-K3H3-D-N-6-IN,air cylinder,NaN,NaN,cylinder,NaN,In,YA2Z Shiga
...,...,...,...,...,...,...,...,...,...,...,...,...
481121,61454685,TO_REVIEW,SKF,NaN,NU 308 ECMP/C3,"Rodamiento SKF, NU 308 ECMP/C3","Rodamiento SKF, NU 308 ECMP/C3",NaN,Rodamiento,NaN,NaN,QF13
481122,61454684,SPARROW_APPROVED,SKF,NaN,6308/C3,Rillenkugellager,"Rodamiento SKF, ref. 6308 C3",NaN,Rillenkugellager,NaN,NaN,QF13
481123,61454683,SPARROW_APPROVED,SKF,1206KTN9/C3,NaN,Zweireihiges Pendelkugellager ohne Dichtung,"Rodamiento SKF,1206 KTN9/C3",NaN,PENDELKUGELLAGER,NaN,NaN,QF13
481124,61454682,CLARIFICATION_NEEDED,SKF,NaN,NaN,"Rodamiento SKF,6218","Rodamiento SKF,6218",NaN,Rodamiento,NaN,NaN,QF13


### 2) Standarization

In [27]:
manufacturer_names = df_bayer['manufacturer_name'].drop_duplicates().to_frame(name="Original Name")

In [42]:
standardize = standardize_manufacturers_df(manufacturer_names.iloc[0:100], input_col="Original Name", output_col="Standardized Name")

In [50]:
standardize

,Original Name,Standardized Name,processed_standardized_name,processed_original_name
0,CKD,CKD,ckd,ckd
23,Mitsubishi Electric,Mitsubishi,mitsubishi,mitsubishi electric
35,Habasit,Habasit,habasit,habasit
42,Nordson,Nordson,nordson,nordson
54,CONVUM,CONVUM,convum,convum
...,...,...,...,...
2180,M.SYTEM,M.SYTEM,m sytem,m sytem
2185,yokogawa,Yokogawa,yokogawa,yokogawa
2196,kuroda,Kuroda,kuroda,kuroda
2198,Koganei,Koganei,koganei,koganei


For festo

In [51]:
festo_df = manufacturer_names[manufacturer_names["Original Name"].astype(str).str.contains("Festo", case=False, na=False)]
results_festo = standardize_manufacturers_df(festo_df, input_col="Original Name", output_col="Standardized Name")
results_festo

,Original Name,Standardized Name
1180,FESTO,FESTO
2506,Festo AG & CO. KG,Festo
2551,Festo,Festo
6974,Festo Inc.,Festo
7278,Festo Corporation,Festo
99390,Festo / Pester Pac Automation,Festo
99735,Festo AG & Co. KG,Festo
152481,FESTO CORP.,Festo
391240,Festo/Uhlmann,Festo
434712,Festo AG & Co.,Festo


In [52]:
df_lfa_manufacturer=df_lfa['MCOD1']
df_lfa_manufacturer.head()

0    AAA TECHNOLOGY & SPECIALT
1            ABCO PLASTICS DIV
2             PIKOTEK/ACCUSEAL
3         ACF INDUSTRIES, INC.
4    ATLANTIC STATES CAST IRON
Name: MCOD1, dtype: object

Preprocessing

In [53]:
# Preprocess names in both DataFrames
df_lfa['processed_name'] = df_lfa['MCOD1'].apply(preprocess_name)
standardize['processed_standardized_name'] = standardize['Standardized Name'].apply(preprocess_name)
standardize['processed_original_name'] = standardize['Original Name'].apply(preprocess_name)

In [54]:
standardize

,Original Name,Standardized Name,processed_standardized_name,processed_original_name
0,CKD,CKD,ckd,ckd
23,Mitsubishi Electric,Mitsubishi,mitsubishi,mitsubishi electric
35,Habasit,Habasit,habasit,habasit
42,Nordson,Nordson,nordson,nordson
54,CONVUM,CONVUM,convum,convum
...,...,...,...,...
2180,M.SYTEM,M.SYTEM,m sytem,m sytem
2185,yokogawa,Yokogawa,yokogawa,yokogawa
2196,kuroda,Kuroda,kuroda,kuroda
2198,Koganei,Koganei,koganei,koganei


### 3) Matching

#### 1) Substring matching

In [126]:
standardize = standardize.dropna()  # 1 row with NaN values

In [131]:
standardize[standardize['processed_original_name']=='nippon heater']

,Original Name,Standardized Name,processed_standardized_name,processed_original_name
608,Nippon Heater,Nippon,nippon,nippon heater


In [132]:
df_lfa[df_lfa['NAME1']=='NIPPON STEEL CORP']

,OPSYS,LIFNR,LAND1,NAME1,ADRNR,MCOD1,KTOkK,LOEVM,processed_name,matched_standardized_name,matched_non_standardized_name
464,PMD,5373319,KY,NIPPON STEEL CORP,446318,NIPPON STEEL CORP,YCMP,NaN,nippon steel corp,nippon,


"nippon steel corp" does NOT appear as a substring within "nippon heater".

"nippon heater" does NOT appear as a substring within "nippon steel corp".

This is the substring matching

In [103]:
df_lfa

,OPSYS,LIFNR,LAND1,NAME1,ADRNR,MCOD1,KTOkK,LOEVM,processed_name,matched_standardized_name,matched_non_standardized_name
0,PMD,5368080,KY,AAA Technology & Specialties Co,445840,AAA TECHNOLOGY & SPECIALT,YCMP,NaN,aaa technology specialt,None,
1,PMD,5368099,KY,ABCO PLASTICS DIV,445835,ABCO PLASTICS DIV,YCMP,NaN,abco plastics div,None,
2,PMD,5368102,KY,Pikotek/AccuSeal,445813,PIKOTEK/ACCUSEAL,YCMP,NaN,pikotek accuseal,None,
3,PMD,5368110,KY,"ACF INDUSTRIES, INC.",445841,"ACF INDUSTRIES, INC.",YCMP,NaN,acf industries inc,None,
4,PMD,5368129,KY,Atlantic States Cast Iron Pipe C,445777,ATLANTIC STATES CAST IRON,YCMP,NaN,atlantic states cast iron,None,
...,...,...,...,...,...,...,...,...,...,...,...
10978,PMD,8562020,KY,CORROSION PRODUCTS OF TEXAS,1321945,CORROSION PRODUCTS OF TEX,YCMP,NaN,corrosion products of tex,None,
10979,PMD,8562055,KY,"FUJIKIN OF AMERICA,INC.",1321946,"FUJIKIN OF AMERICA,INC.",YCMP,NaN,fujikin of america inc,fuji,
10980,PMD,8568363,KY,"TRD MANUFACTURING,INC.",1324328,"TRD MANUFACTURING,INC.",YCMP,NaN,trd manufacturing inc,None,
10981,PMD,8575734,KY,COOPER WIRING DEVICES,1325920,COOPER WIRING DEVICES,YCMP,NaN,cooper wiring devices,None,


In [133]:
df_lfa['matched_standardized_name'] = df_lfa['processed_name'].map(lambda x: next((s for s in standardize['processed_standardized_name'] if x in s or s in x), None))
df_lfa['matched_non_standardized_name'] = df_lfa['processed_name'].map(lambda x: next((s for s in standardize['processed_original_name'] if x in s or s in x), None))

In [134]:
standardize['processed_original_name'].value_counts()

processed_original_name
bando                        2
keyence                      2
panasonic                    2
fuji electric                2
omron                        2
                            ..
pisco                        1
sanwa shutter corporation    1
nihon heater                 1
nippon heater                1
hisaka                       1
Name: count, Length: 88, dtype: int64

In [135]:
df_lfa['matched_non_standardized_name'].value_counts()

matched_non_standardized_name
continental            22
hitachi                 5
orion                   4
sinfonia technology     3
smc                     3
nsd                     3
mitsubishi              3
ntn                     2
tsubaki                 2
yokogawa                2
nsk                     2
nok                     2
koyo                    2
gates unitta asia       2
sanyo                   2
schneider electric      2
panasonic               2
ckd                     2
taiyo                   2
toshiba                 2
asahi electronics       1
nordson                 1
nidec                   1
mitsubishi electric     1
forbo                   1
tdk                     1
habasit                 1
fuji electric           1
isuzu                   1
omron                   1
idec                    1
festo                   1
pisco                   1
bando                   1
keyence                 1
thk                     1
moritex                 1
oriental

x in s → The value from df_lfa['processed_name'] is a substring of the value from the standardize list.

s in x → The value from the standardize list is a substring of the value from df_lfa['processed_name'].

In [136]:
# df_lfa['matched_standardized_name'] = df_lfa['processed_name'].map(
#     lambda x: next(
#         (
#             s for s in standardize['processed_standardized_name']
#             if x in s or s in x or any(word in s.split() for word in x.split())
#         ),
#         None
#     )
# )
# df_lfa['matched_non_standardized_name'] = df_lfa['processed_name'].map(
#     lambda x: next(
#         (
#             s for s in standardize['processed_original_name']
#             if x in s or s in x or any(word in s.split() for word in x.split())
#         ),
#         None
#     )
# )

In [137]:
# # Merge based on the processed names (approximate matching)
# df_lfa['matched_standardized_name'] = df_lfa['processed_name'].map(lambda x: next((s for s in standardized['processed_standardized_name'] if x in s or s in x), None))

In [138]:
df_lfa_no_dup = df_lfa.drop_duplicates(subset=['processed_name'], keep='first')

Taking both matches

In [140]:
df_lfa_filtered = df_lfa_no_dup[(df_lfa_no_dup['matched_standardized_name'].notna()) | (df_lfa_no_dup['matched_non_standardized_name'].notna())]
df_lfa_filtered

,OPSYS,LIFNR,LAND1,NAME1,ADRNR,MCOD1,KTOkK,LOEVM,processed_name,matched_standardized_name,matched_non_standardized_name
53,PMD,5368684,KY,ASAHI/AMERICA,445903,ASAHI/AMERICA,YCMP,NaN,asahi america,asahi,None
94,PMD,5369133,KY,BUSSMANN DIV./COOPER IND,445946,BUSSMANN DIV./COOPER IND,YCMP,NaN,bussmann div cooper ind,buss,None
149,PMD,5369753,KY,"CONTINENTAL MFG,INC",445988,"CONTINENTAL MFG,INC",YCMP,NaN,continental mfg inc,continental,continental
262,PMD,5371030,KY,GATES,446111,GATES,YCMP,NaN,gates,gates unitta,gates unitta asia
464,PMD,5373319,KY,NIPPON STEEL CORP,446318,NIPPON STEEL CORP,YCMP,NaN,nippon steel corp,nippon,None
...,...,...,...,...,...,...,...,...,...,...,...
10720,PMD,8097998,KY,BUSS-SMS-CANZLER,99888769,BUSS-SMS-CANZLER,YCMP,NaN,buss sms canzler,buss,None
10845,PMD,8098321,KY,OHMEX,99889403,OHMEX,YCMP,NaN,ohmex,ohm,None
10923,PMD,8098678,KY,TECHNO,99890343,TECHNO,YCMP,NaN,techno,None,sinfonia technology
10950,PMD,8247931,KY,FREUDENBERG-NOK,1238145,FREUDENBERG-NOK,YCMP,NaN,freudenberg nok,nok,nok


In [141]:
# df_lfa_filtered.to_excel('df_matched.xlsx')

In [142]:
matched_words = set()
for name in df_lfa['matched_standardized_name'].dropna().unique():
    matched_words.update(name.split())  # Split into words and add to the set

# Extract all standardized names
standardize_values = standardize['processed_standardized_name'].unique()

# Identify unmatched values where no single word was matched
unmatched_values = [
    val for val in standardize_values
    if not any(word in matched_words for word in val.split())
]


In [143]:
len(unmatched_values)

40

In [144]:
unmatched_values

['convum',
 'koganei',
 'maruyasu',
 'kyoto',
 'yasukawa',
 'omori',
 'misumi',
 'sankyo',
 'kintsune',
 'nihon',
 'sanwa',
 'sunmulon',
 'iai',
 'mitsuboshi',
 'kofloc',
 'hasegawa',
 'mankasi',
 'denson',
 'polimax',
 'kyoto seisakusyo',
 'kyowa',
 'anritsu',
 'miki pulley',
 'nihon densan',
 'xks',
 'nichiyasu',
 'sinfonia',
 'futaku',
 'beltcon',
 'sanei',
 'tecweigh',
 'nitta',
 'toyox',
 'maruha',
 'azbil',
 'yamatake',
 'yasugkawa',
 'm sytem',
 'kuroda',
 'hisaka']

In [145]:
df_lfa_filtered

,OPSYS,LIFNR,LAND1,NAME1,ADRNR,MCOD1,KTOkK,LOEVM,processed_name,matched_standardized_name,matched_non_standardized_name
53,PMD,5368684,KY,ASAHI/AMERICA,445903,ASAHI/AMERICA,YCMP,NaN,asahi america,asahi,None
94,PMD,5369133,KY,BUSSMANN DIV./COOPER IND,445946,BUSSMANN DIV./COOPER IND,YCMP,NaN,bussmann div cooper ind,buss,None
149,PMD,5369753,KY,"CONTINENTAL MFG,INC",445988,"CONTINENTAL MFG,INC",YCMP,NaN,continental mfg inc,continental,continental
262,PMD,5371030,KY,GATES,446111,GATES,YCMP,NaN,gates,gates unitta,gates unitta asia
464,PMD,5373319,KY,NIPPON STEEL CORP,446318,NIPPON STEEL CORP,YCMP,NaN,nippon steel corp,nippon,None
...,...,...,...,...,...,...,...,...,...,...,...
10720,PMD,8097998,KY,BUSS-SMS-CANZLER,99888769,BUSS-SMS-CANZLER,YCMP,NaN,buss sms canzler,buss,None
10845,PMD,8098321,KY,OHMEX,99889403,OHMEX,YCMP,NaN,ohmex,ohm,None
10923,PMD,8098678,KY,TECHNO,99890343,TECHNO,YCMP,NaN,techno,None,sinfonia technology
10950,PMD,8247931,KY,FREUDENBERG-NOK,1238145,FREUDENBERG-NOK,YCMP,NaN,freudenberg nok,nok,nok


In [68]:
# df_lfa_filtered.to_excel('df_matched.xlsx')

In [69]:
# df_lfa_filtered[df_lfa_filtered['matched_standardized_name']=='continental']

In [146]:
df_lfa_filtered[df_lfa_filtered["MCOD1"].astype(str).str.contains("Festo", case=False, na=False)]

,OPSYS,LIFNR,LAND1,NAME1,ADRNR,MCOD1,KTOkK,LOEVM,processed_name,matched_standardized_name,matched_non_standardized_name
2793,PMD,7125681,KY,FESTO,716526,FESTO,YCMP,NaN,festo,festo,festo


In [71]:
# duplicates_df = df_lfa[df_lfa.duplicated(subset=['MCOD1'], keep=False)]
# duplicates_df

In [147]:
df_lfa_filtered.matched_standardized_name.value_counts()

matched_standardized_name
continental           22
ohm                   11
fuji                   7
hitachi                5
buss                   5
mitsubishi             4
orion                  3
sato                   3
nsd                    3
nippon                 3
smc                    3
ntn                    2
sanyo                  2
koyo                   2
tsubaki                2
nsk                    2
nok                    2
panasonic              2
yokogawa               2
schneider electric     2
taiyo                  2
toshiba                2
ckd                    2
nkk                    2
gates unitta           2
isuzu                  1
nordson                1
nidec                  1
tdk                    1
habasit                1
denso                  1
asahi                  1
keyence                1
omron                  1
idec                   1
festo                  1
pisco                  1
bando                  1
thk                    1

In [148]:
# Count None (or NaN) values in the specified columns
none_counts = df_lfa[['matched_standardized_name', 'matched_non_standardized_name']].isna().sum()

# Display the results
none_counts

matched_standardized_name        10866
matched_non_standardized_name    10892
dtype: int64

### Benchmarks

For matched nouns

In [149]:
df_matched=pd.read_excel('df_matched.xlsx')    # Benchmark the matched ones

In [150]:
(82/104)*100   # Accuracy

78.84615384615384

For unmatched nouns

Reviewing the names, just 2 of the them appear in the LFA table:  'nippon heater' and 'asahi kasei',

In [149]:
unmatched_values

['myotoku',
 'koganei',
 'maruyasu',
 'kyoto seisakusho',
 'omori',
 'asahi kasei',
 'misumi',
 'sankyo',
 'kintsune',
 'nippon heater',
 'nihon heater',
 'sanwa',
 'iai',
 'mitsuboshi',
 'kofloc',
 'sato parts',
 'hasegawa iron',
 'mankasi seisakusho',
 'polimax belt',
 'gs yuasa',
 'kyoto seisakusyo',
 'kyowa',
 'anritsu',
 'miki pulley',
 'nihon densan shinpo',
 'xks',
 'nichiyasu',
 'omori machinery co ltd',
 'jtekt',
 'futaku',
 'beltcon',
 'toyox',
 'azbil',
 'tsubakimoto chain',
 'hisaka']

#### 2) Just word-matching. 

Too many matches

In [151]:
df_lfa['matched_standardized_name'] = df_lfa['processed_name'].map(lambda x: word_based_match(x, standardize['processed_standardized_name']))
df_lfa['matched_non_standardized_name'] = df_lfa['processed_name'].map(lambda x: word_based_match(x, standardize['processed_original_name']))

In [152]:
df_lfa

,OPSYS,LIFNR,LAND1,NAME1,ADRNR,MCOD1,KTOkK,LOEVM,processed_name,matched_standardized_name,matched_non_standardized_name
0,PMD,5368080,KY,AAA Technology & Specialties Co,445840,AAA TECHNOLOGY & SPECIALT,YCMP,NaN,aaa technology specialt,None,sinfonia technology
1,PMD,5368099,KY,ABCO PLASTICS DIV,445835,ABCO PLASTICS DIV,YCMP,NaN,abco plastics div,None,None
2,PMD,5368102,KY,Pikotek/AccuSeal,445813,PIKOTEK/ACCUSEAL,YCMP,NaN,pikotek accuseal,None,None
3,PMD,5368110,KY,"ACF INDUSTRIES, INC.",445841,"ACF INDUSTRIES, INC.",YCMP,NaN,acf industries inc,None,buss inc usa
4,PMD,5368129,KY,Atlantic States Cast Iron Pipe C,445777,ATLANTIC STATES CAST IRON,YCMP,NaN,atlantic states cast iron,None,hasegawa iron
...,...,...,...,...,...,...,...,...,...,...,...
10978,PMD,8562020,KY,CORROSION PRODUCTS OF TEXAS,1321945,CORROSION PRODUCTS OF TEX,YCMP,NaN,corrosion products of tex,None,None
10979,PMD,8562055,KY,"FUJIKIN OF AMERICA,INC.",1321946,"FUJIKIN OF AMERICA,INC.",YCMP,NaN,fujikin of america inc,None,buss inc usa
10980,PMD,8568363,KY,"TRD MANUFACTURING,INC.",1324328,"TRD MANUFACTURING,INC.",YCMP,NaN,trd manufacturing inc,None,buss inc usa
10981,PMD,8575734,KY,COOPER WIRING DEVICES,1325920,COOPER WIRING DEVICES,YCMP,NaN,cooper wiring devices,None,None


#### 3) Matching based on similarity

Difficult to review and to stablish the threshold

In [153]:
def find_best_match(name, candidates):
    """Finds the best match from a list using hybrid similarity."""
    best_match, best_score = None, 0
    
    for candidate in candidates:
        score = hybrid_similarity(name, candidate)
        if score > best_score:
            best_match, best_score = candidate, score
    
    return best_match, best_score

In [154]:
df_lfa[['best_standardized_match', 'standardized_similarity']] = df_lfa['processed_name'].apply(
    lambda x: find_best_match(x, standardize['processed_standardized_name'])
).apply(pd.Series)

df_lfa[['best_non_standardized_match', 'non_standardized_similarity']] = df_lfa['processed_name'].apply(
    lambda x: find_best_match(x, standardize['processed_original_name'])
).apply(pd.Series)

In [155]:
df_lfa

,OPSYS,LIFNR,LAND1,NAME1,ADRNR,MCOD1,KTOkK,LOEVM,processed_name,matched_standardized_name,matched_non_standardized_name,best_standardized_match,standardized_similarity,best_non_standardized_match,non_standardized_similarity
0,PMD,5368080,KY,AAA Technology & Specialties Co,445840,AAA TECHNOLOGY & SPECIALT,YCMP,NaN,aaa technology specialt,None,sinfonia technology,kyoto seisakusyo,0.216,sinfonia technology,0.442000
1,PMD,5368099,KY,ABCO PLASTICS DIV,445835,ABCO PLASTICS DIV,YCMP,NaN,abco plastics div,None,None,habasit,0.252,sato parts,0.264000
2,PMD,5368102,KY,Pikotek/AccuSeal,445813,PIKOTEK/ACCUSEAL,YCMP,NaN,pikotek accuseal,None,None,kyoto seisakusyo,0.264,kyoto seisakusho,0.264000
3,PMD,5368110,KY,"ACF INDUSTRIES, INC.",445841,"ACF INDUSTRIES, INC.",YCMP,NaN,acf industries inc,None,buss inc usa,anritsu,0.240,buss inc usa,0.362000
4,PMD,5368129,KY,Atlantic States Cast Iron Pipe C,445777,ATLANTIC STATES CAST IRON,YCMP,NaN,atlantic states cast iron,None,hasegawa iron,gates unitta,0.192,hasegawa iron,0.302000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10978,PMD,8562020,KY,CORROSION PRODUCTS OF TEXAS,1321945,CORROSION PRODUCTS OF TEX,YCMP,NaN,corrosion products of tex,None,None,moritex,0.228,moritex,0.228000
10979,PMD,8562055,KY,"FUJIKIN OF AMERICA,INC.",1321946,"FUJIKIN OF AMERICA,INC.",YCMP,NaN,fujikin of america inc,None,buss inc usa,fuji,0.186,fuji electric,0.306000
10980,PMD,8568363,KY,"TRD MANUFACTURING,INC.",1324328,"TRD MANUFACTURING,INC.",YCMP,NaN,trd manufacturing inc,None,buss inc usa,mankasi,0.216,buss inc usa,0.260000
10981,PMD,8575734,KY,COOPER WIRING DEVICES,1325920,COOPER WIRING DEVICES,YCMP,NaN,cooper wiring devices,None,None,nihon densan,0.216,schneider electric,0.216000
